## Название документа

Название НПА – это краткий текст, описывающий основную суть и тему документа.
Название часто начинается с предлогов «о» или «об». В тексте НПА название может встречаться
в кавычках или без кавычек. Название не включает в себя тип НПА.

Для примера название документа – это «О заместителе Министра внутренних дел
Российской Федерации – начальнике Следственного департамента Министерства внутренних дел Российской Федерации».

Поле name должно содержать строку с названием в кавычках или без, в зависимости от того, как оно встретилось в тексте НПА. При сравнении предсказанного и настоящего номера в проверяющей системе не учитывается регистр букв, пробелы и символы перевода строки не различаются.

## Орган, принявший акт

Часто автор документа указывается после слова «Принят», но далеко не всегда, например в указе автор - «Президент Российской Федерации».

В рамках задания необходимо для каждого документа определить орган, его принявший. Наименование органов необходимо извлекать в «нормализованном виде»:

* Ярославская областная дума
* Федеральная служба по надзору в сфере образования и науки
* Правительство Удмуртской Республики
* Департамент охраны окружающей среды и природопользования Ярославской области

В обучающей и тестовой выборке около 200 уникальных органов. В тестовой выборке есть органы, которых нет в обучающей.

Поле authority должно содержать наименование органа в нормализованном виде. При сравнении предсказанного и настоящего органа не учитывается регистр букв, пробелы и символы перевода строки не различаются.

### Оценка качества

Допускается нечеткое совпадение. Метрика качества – средний коэффициент Жаккара.

Коэффициент Жаккара вычисляется следующим образом:

$$ L(s_{extr}, s_{gold}) = \frac{intersect}{offset_{start} + intersect + offset_{end}} $$

где $intersect$ - длина самой длинной общей подстроки у выделенной строки и строки из «золотой метки», $offset_{start}$ - наибольшее количество символов до начала общей подстроки в выделенной строке и в строке из «золотой метки», $offset_{end}$ - наибольшее количество символов от конца общей подстроки до конца строки в выделенной строке и в строке из «золотой метки».

Эта метрика принимает значения от 0 до 1 и штрафует как за выделение более короткой строки чем строка из «золотой метки», так и за выделение более длинной строки.

* указ - президент, глава, губернатор

* федеральный закон - Государственная Дума Федерального собрания Российской Федерации

* приказ - управление, министерство, департамент, федеральная служба, комитет, агентство

* распоряжение - правительство, администрация, губернатор, президент, глава, дума

* закон - Государственный Совет, Законодательное собрание, областная дума, Законодательная Дума, Государственный Совет, Народное Собрание, Эл Курултай, Президент, областное Собрание

* постановление - Правительство Российской Федерации, Конституционный Суд, Кабинет министров, 
    Правительство, Администрация, Губернатор, Глава, областной совет, областное Собрание,
    Законодательное собрание

In [1]:
import re
from eval_module import quality
import json
import numpy as np

In [2]:
train = []
labeled = []
names = []
with open("train/gold_labels.txt", "r") as read_file:
    for doc_info in read_file.readlines():
        doc_dict = json.loads(doc_info)
        docname = 'train/txts/' + doc_dict['id'] + '.txt'
        names.append(docname)
        with open(docname, 'r') as f:
            train.append(f.read())
        labeled.append(doc_dict['label'])

true_types = np.array([x['type'] for x in labeled])

In [3]:
PREPR_EXPR = re.compile(r'[^\wа-я"\s\.,;:№\-\)\(]')

def preprocess_doc(doc):
    doc = '\n' + doc.lower()
    doc = re.sub(r"[пнл]оста[нв]ов[пнл]е[нпл]и[ек]", r"постановление", doc)
    doc = re.sub(r"[^\s]+дл.вско[йи]", "свердловской", doc)
    doc = re.sub(r"[а-я]*нене?цко?го", "ненецкого", doc)
    doc = re.sub(r'[^\s]?при[нвп]ят', 'принят', doc)
    doc = re.sub('_', ' ', doc)
    doc = re.sub(' +', ' ', doc)
    return PREPR_EXPR.sub('', doc)

In [4]:
NAME_EXPR = re.compile(r'(\nоб?\s[а-я]([^\n]+\n)+\n)')
UKAZ_EXPR = re.compile(r'(\nвопросы\s([^\n]+\n)+\n)')
POST_EXPR = re.compile(r'"о\s[^"]+"')
LAST_EXPR = re.compile(r'(об? ([^\n]+\n)+)\n')
RASP_EXPR = re.compile(r'\nоб? [а-я ]+ [а-я]\. ?[а-я]\.\n')
POST2_EXPR = re.compile(r'(о внесении изменени[яй] [^\"]+)\"')


def extract_name(doc, doc_type):
    doc = preprocess_doc(doc)
    if doc_type == "распоряжение":
        match = RASP_EXPR.search(doc)
        if match:
            return match.group().strip()
    match = NAME_EXPR.search(doc)
    if match:
        name = match.group(0).strip()
        if doc.find('правительство пензенской области') != -1:
            match = POST2_EXPR.search(name)
            if match:
                return match.group(0).strip()
        if doc_type == 'закон':
            ind = name.find("принят")
            if ind != -1:
                name = name[:ind].strip()
        return name
    if doc_type == 'указ':
        match = UKAZ_EXPR.search(doc)
        if match:
            return match.group(0).strip()
    match = POST_EXPR.search(doc)
    if match:
        return match.group()
    match = LAST_EXPR.search(doc)
    if match:
        return match.group(0).strip()
    return ""

In [5]:
subs = {
    "губернатора": "губернатор",
    "конституционного суда": "конституционный суд",
    "правительства": "правительство",
    "главы": "глава",
    "администрации": "администрация",
    "президента": "президент",
}

POST_EXPR_1 = re.compile(r'(([а-я\s?]+\n)?[^\n]+(области|края|российской федерации|совет депутатов|'
                         r'округа))\s+([^\n]+созыва\s+)?постановление\s*\n')
POST_EXPR_2 = re.compile(r'(правительство\n?[^\n]+)\s+постановление\s*\n')
POST_EXPR_3 = re.compile(r'постановление\s+(губернатора|конституционного суда|'
                         r'правительства|главы|администрации)\s([а-я ]+)')
POST_EXPR_4 = re.compile(r'([^\n]+) постановляет')
G_EXPR = re.compile(r'(правительство\s([^\n]+(республики|области|федерации|края)|республики[^\n]+))')

patterns = [
    ("г. воронеж", "правительство воронежской области"),
    ("адыгея", "Кабинет министров Республики Адыгея"),
    ("бурятия", "Правительство Республики Бурятия"),
    ("законодательное собрание владимирской области", "Законодательное собрание Владимирской области"),
    ("мирской области", "Администрация Владимирской области"),
    ("ямало-ненецкого", "Правительство Ямало-Ненецкого автономного округа"),
    ("кабинет министров", "Кабинет министров Чувашской Республики")
]

def post_authority(doc):
    for pattern in patterns:
        if doc.find(pattern[0]) != -1:
            return pattern[1]
    if doc.startswith('\nправительство'):
        first_line = doc.split('\n')[1].strip()
        if len(first_line.split(' ')) > 2:
            return first_line

    match = G_EXPR.search(doc)
    if match:
        return match.group(1)
    match = POST_EXPR_1.search(doc)
    if match:
        return match.group(1).strip()
    match = POST_EXPR_2.search(doc)
    if match:
        return match.group(1).strip()
    match = POST_EXPR_3.search(doc)
    if match:
        return subs[match.group(1)] + " " + match.group(2).strip()
    match = POST_EXPR_4.search(doc)
    if match and len(match.group(1).split(' ')) > 2:
        authority = match.group(1).strip()
        authority = re.sub(r'(от )?\d.*', '', authority)
        if len(authority.split(' ')) < 6:
            return authority
    
    return ""

In [64]:
ZAKON_EXPR = re.compile(
    r'принят\s([^\n]*(государственн|совет|законодательн|собрание|'
    r'областн|дум|народн|курултай|президент)[^\n\d]*(\n[а-я\-]+ \d\d)?)')
ZAKON_EXPR_2 = re.compile(r'\n([а-я\s]+)\s[^\s]*постановляет')
ZAKON_EXPR_3 = re.compile(r'\nзакон\n\n([а-я ]+)\n\n')
ZAKON_EXPR_4 = re.compile(r'принят\s([а-я\- ]+\s([а-я\- ]+\s)?)\d\d')

lemmatizer = {
    'законодательным': 'законодательное', 'областной': 'областная', 'думой': 'дума',
    'народным': 'народное', 'президентом': 'президент', 'государственным': 'государственный',
    'областное': 'областным', 'законодательной': 'законодательная',
    'собранием': 'собрание', 'советом': 'совет',
}


def normalize(authority):
    tokens = re.sub(r'[^а-я \-]', ' ', authority).split(' ')
    authority = ""
    for token in tokens:
        if token in lemmatizer:
            authority += lemmatizer[token] + ' '
        else:
            authority += token + ' '
    authority = re.sub(r'ой( [а-я\-]* ?дума)', r'ая\1', authority)
    return authority.strip()

def zakon_authority(doc):
    if doc.startswith('\nкостромская областная дума'):
        return 'костромская областная дума'
    
    match = ZAKON_EXPR_3.search(doc)
    if match and doc.find('принят законодательным собранием') != -1:
        return 'законодательное собрание ' + match.group(1).strip()
    
    
    match = ZAKON_EXPR_2.search(doc)
    if match:
        return match.group().strip()
    match = ZAKON_EXPR_3.search(doc)
    if match:
        return 'законодательная дума ' + match.group(1).strip()
    match = ZAKON_EXPR_4.search(doc)
    if match:
        return normalize(match.group(1))

    match = ZAKON_EXPR.search(doc)
    if not match:
        if not match:
            return ""
    
    return normalize(match.group(1))


In [65]:
def ukaz_authority(doc):
    variants = []
    match = re.search(r'президента\s+российской\s+федерации', doc)
    if match:
        variants.append(('президент российской федерации', match.span()[0]))

    match = re.search(r'губернатора? ', doc)
    if match:
        index = match.span()[0]
        doc1 = doc[index:]
        authority = doc1.split('\n')[0]
        authority = 'губернатор ' + ' '.join(authority.split(' ')[1:])
        variants.append((authority, index))
    index = doc.find('главы')
    if index != -1:
        doc1 = doc[index:]
        authority = doc1.split('\n')[0]
        authority = 'глава ' + ' '.join(authority.split(' ')[1:])
        variants.append((authority, index))
    match = re.search(r'глава\s*\n', doc)
    if match:
        index = match.span()[0]
        doc1 = doc[index:]
        authority = 'глава ' + doc1.split('\n')[1]
        variants.append((authority, index))
    for author in ['глава ', 'губернатор\n']:
        index = doc.find(author)
        if index != -1:
            doc1 = doc[index:]
            authority = author + doc1.split('\n')[1]
            variants.append((authority, index))
    if variants:
        authority = variants[0][0]
        index = variants[0][1]
        for a, i in variants:
            if i < index:
                authority = a
                index = i
        return authority
    return ""

In [66]:
PRIKAZ_EXPR = re.compile(r'\n([^\n]*(управление|министерство|департамент|'
                         r'федеральная|служба|комитет|агентство)([^\n]+\n)*)\n')

def prikaz_authority(doc):
    match = PRIKAZ_EXPR.search(doc)
    if match:
        return match.group(1)
    return ""

In [67]:
organizations = re.compile(r'правительств[оа]|министерство|администраци[яи]|управление|' 
                           'комитет|департамент|президента?|губернатора?|главы|'
                           'законодательное собрание|кабинет министров|областное собрание|'
                           'конституционный суд|конституционного суда|государственная дума|дума')

def extract_authority(doc, doc_type):
    if doc_type == 'федеральный закон':
        return "Государственная Дума Федерального собрания Российской Федерации"
    doc = preprocess_doc(doc)
    if doc_type == 'постановление':
        authority = post_authority(doc)
        if authority:
            return authority
    elif doc_type == 'приказ':
        authority = prikaz_authority(doc)
        if authority:
            return authority
    elif doc_type == 'закон':
        authority = zakon_authority(doc)
        if authority:
            return authority
    if doc_type == 'указ':
        authority = ukaz_authority(doc)
        if authority:
            return authority
    authority = ""
    matches = organizations.findall(doc)
    if matches:
        organization = matches[0]
        index = doc.find(organization)
        doc = doc[index:]
        authority = doc.split('\n')[0]
        if authority == organization:
            authority += ' ' + doc.split('\n')[1]
        for sub in subs: 
            if authority.startswith(subs[sub]):
                authority = re.sub(sub, subs[sub], authority)
        authority = re.sub(r'(от )?\d.*', '', authority).strip()
        return authority
    index = doc.find('принят ')
    if index != -1:
        doc = doc[index + len('принят'):]
        authority = doc.split('\n')[0]
        authority = re.sub(r'(от )?\d.*', '', authority)

    return authority

In [68]:
def predict(test, doc_types):
    results = []
    for doc, doc_type in zip(test, doc_types):
        authority = extract_authority(doc, doc_type)
        name = extract_name(doc, doc_type)
        prediction = {"type": "",
                      "date": "",
                      "number": "",
                      "authority": authority,
                      "name": name}
        results.append(prediction)
    return results

In [69]:
predicted = predict(train, true_types)
quality(predicted, labeled)

{'date_accuracy': 0.0,
 'number_accuracy': 0.0,
 'type_f1_score': 0.0,
 'name_jaccard': 0.9829125327,
 'authority_jaccard': 0.9775330141,
 'subtasks_improves': 2}

In [70]:
predicted_labels = np.array(predicted)
test_labels = np.array(labeled)
names = np.array(names)
for doc_type in ['федеральный закон', 'постановление', 'приказ', 'распоряжение','закон', 'указ']:
    ids = np.where(true_types == doc_type)
    print(f"{doc_type}({ids[0].shape[0]}): {quality(predicted_labels[ids], test_labels[ids])['authority_jaccard']}")

федеральный закон(192): 1.0
постановление(2459): 0.9875189695
приказ(77): 0.6794957433
распоряжение(430): 0.9944086663
закон(199): 0.8692417746
указ(632): 0.9907817016


In [72]:
from eval_module import preprocess, string_jaccard_metric

doc_type = 'закон'
ids = np.where(true_types == doc_type)

y_test = [(x['authority'], x['type']) for x in test_labels[ids]]
y_pred = [x['authority'] for x in predicted_labels[ids]]

wrong_docs = []
i = 0
for test, pred in zip(y_test, y_pred):
    test_, pred_ = preprocess(test[0]), preprocess(pred)
    if string_jaccard_metric([test_], [pred_]) < 0.6:
        wrong_docs.append((names[ids][i], test, pred))
    i += 1
    
for doc_name, test, pred in wrong_docs:
    with open(doc_name, 'r') as f:
        print(f"right value = {test}")
        print(f"wrong value = {pred}")
        print(f"docname = {doc_name}")
        print(f.read())
        print("===============================================================================================")        
        print("===============================================================================================")

right value = ('Законодательная Дума Томской области', 'закон')
wrong value = законодательная дума
docname = train/txts/c9ddd40b55f0ba507df55040aca8d5fbd26b34f4.txt
ЗАКОН ТОМСКОЙ ОБЛАСТИ

©О внесении изменений в статьи 4 и 7 Закона Томской
области "О премиях Томской области в сфере
образования, науки, здравоохранения и культуры"

Принят Законодательной Думой 26 февраля 2015 года
Томской области (постановление № 2511)
Статья 1

Внести в Закон Томской области от 13 марта 2006 года № 29-О3
"О премиях Томской области в сфере образования, науки, здравоохранения и
культуры" (Официальные ведомости Государственной Думы Томской области,
2006, № 51 (112), постановление от 28.02.2006 № 2870; № 52 (113)-П,
постановление от 30.03.2006 № 2979; № 53 (114), постановление от 25.05.2006
№ 3052; 2007, № 6 (128), постановление от 26.07.2007 № 400; Официальные
ведомости Законодательной Думы Томской области, 2011, № 47 (169),
постановление от 31.03.2011 № 4160; 2012, № 13 (189), постановление от
25.10.2012 